In [2]:
from ase.io import read, write
from ase.visualize import view
from ase import Atoms
import numpy as np

def shrink_c_axis_to_atoms(atoms, padding=0.0):
    """
    원자들이 점유하는 공간만큼만 c축의 길이를 축소합니다.
    
    Args:
        atoms: ASE Atoms 객체
        padding: 원자 범위 양쪽에 추가할 여유 공간 (Angstrom)
    
    Returns:
        c축이 축소된 새로운 Atoms 객체
    """
    atoms = atoms.copy()
    cell = atoms.get_cell()
    a_vec, b_vec, c_vec = cell[0], cell[1], cell[2]
    
    # c축 방향의 단위 벡터 계산
    c_norm = np.linalg.norm(c_vec)
    if c_norm < 1e-8:
        raise ValueError("c축 벡터의 크기가 너무 작습니다.")
    c_unit = c_vec / c_norm
    
    # 모든 원자의 위치를 c축 방향으로 투영
    positions = atoms.get_positions()
    projections = np.dot(positions, c_unit)  # 각 원자의 c축 방향 투영값
    
    # 원자들이 점유하는 범위 계산
    min_proj = np.min(projections)
    max_proj = np.max(projections)
    height = max_proj - min_proj + 2 * padding  # padding 추가
    
    # 새로운 c축 벡터 계산
    new_c_vec = c_unit * height
    
    # 새로운 셀 설정 (원자 위치는 그대로 유지)
    new_cell = np.array([a_vec, b_vec, new_c_vec])
    atoms.set_cell(new_cell, scale_atoms=False)
    
    # 원자들을 새로운 셀의 중심으로 이동 (선택사항)
    # 현재는 원자 위치를 그대로 유지하므로, 필요시 center()를 호출할 수 있습니다
    
    return atoms

patch = read("chunk_results/failure/11_rmsd_0.1225/patch_atoms.cif")
view(patch, viewer='ngl')

In [3]:
# 사용 예시
# padding을 지정하면 원자 범위 양쪽에 여유 공간을 추가할 수 있습니다
patch_shrunk = shrink_c_axis_to_atoms(patch, padding=1.0)  # 양쪽에 5 Angstrom씩 여유 공간

print(f"원본 c축 길이: {np.linalg.norm(patch.get_cell()[2]):.2f} Å")
print(f"축소된 c축 길이: {np.linalg.norm(patch_shrunk.get_cell()[2]):.2f} Å")

# 필요시 원자들을 셀의 중심으로 이동
patch_shrunk.center(axis=2)

view(patch_shrunk, viewer='ngl')


원본 c축 길이: 31.01 Å
축소된 c축 길이: 4.97 Å


In [6]:
print(patch.get_tags())

[0 0 0 0 0 0 0 0]


In [11]:
recon = read("success_cases/mode_1/idx_11/recon_slab.cif")
view(recon, viewer='ngl')